# Tabnet Image Model

In [22]:
import tabnet
from load_data import *
import os
from tqdm import tqdm_notebook
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
tf.compat.v1.enable_eager_execution()


# Display
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [28]:
from load_data_tmp import *

In [29]:
# Load data
X_train, X_test, X_val, y_train, y_test, y_val = load_data_fold(fold=5, drop_id = False)

In [13]:
# load paths and labels
img_folder = "data/images_resized"
img_df = pd.read_csv("data/img_paths.csv")
img_df.reset_index(drop=True, inplace = True)

# load label book
label_cat = ["bathroom", "bedroom", "dining", "hallway", "kitchen", "living"]
label = np.arange(6)
label_book = pd.DataFrame({"label": label_cat, "categorical_label": label})

In [14]:
# prices
url_listing = "http://data.insideairbnb.com/ireland/leinster/dublin/2021-11-07/data/listings.csv.gz"
listings = pd.read_csv(url_listing)
urls = listings["listing_url"]
ids = listings["id"]
price = listings["price"]
price = price.str.replace("$","")
price = price.str.replace(",","")
price = price.astype(float)
listings["price"] = price
listings["log_price"] = np.log(price)
listings = listings[listings["price"]<500]


In [15]:
price_df = listings[["log_price", "id"]]
df = pd.merge(img_df, price_df, on = "id", how = "left")
df.head()

,img_path,id,img_no,label,log_price
0,44077_0.png,44077,0,4.0,4.174387
1,44077_1.png,44077,1,5.0,4.174387
2,44077_2.png,44077,2,1.0,4.174387
3,44077_3.png,44077,3,1.0,4.174387
4,44077_4.png,44077,4,1.0,4.174387


In [16]:
filter = np.any(df.isna(), axis = 1)
df = df[~filter]

# drop "others"
filter = df["label"] == 6.0
df = df[~filter]

In [17]:
bool = []
for id in df["id"]:
    tmp = df[df["id"] == id]
    if len(np.unique(tmp["label"])) >= 4:
        bool.append(True)
    else:
        bool.append(False)

In [18]:
df_new = df[bool]
np.unique(df_new["id"].values).shape
df = df_new

In [19]:
df.shape

(56704, 5)

In [8]:
def input_pipeline(room = 0, df = df):
    ids = []
    features = []
    
    # FILTER DF
    df_room = df[df["label"] == room]
    
    # RESNET
    resnet = tf.keras.applications.resnet.ResNet50(include_top=False, weights='imagenet', pooling="avg", input_shape = (None,None,3))
    resnet_pre = keras.applications.resnet50.preprocess_input
    resnet.trainable = False
    
    for id in tqdm(np.unique(df["id"])):
        filter = df_room["id"] == id  

        try:
            l = []
            if filter.sum() == 0:
                dummy_image = np.zeros((1,256,256,3))
                dummy_image = resnet_pre(dummy_image)
                dummy_image = resnet(dummy_image)
                l.append(dummy_image)                
            else:
                path_id = df_room["img_path"][filter]
                for p in path_id:
                    img_tmp = plt.imread("data/images_resized/"+p)
                    img_tmp = np.expand_dims(img_tmp, axis = 0)
                    img_tmp = resnet_pre(img_tmp)
                    img_tmp = resnet(img_tmp)
                    l.append(img_tmp)
            l = np.stack(l)
            l = np.max(l, axis = 0)
            features.append(l)
            ids.append(id)
        except:
            dummy_image = np.zeros((1,256,256,3))
            dummy_image = resnet_pre(dummy_image)
            dummy_image = resnet(dummy_image)
            l.append(dummy_image)
            l = np.stack(l)
            l = np.max(l, axis = 0)
            features.append(l)
            ids.append(id)
            continue
    features = np.squeeze(np.stack(features))
    filter = np.nonzero(features.sum(axis = 0))[0]
    features = features[:,filter]
    print(len(filter), " features are nonzero.")
    features = features.tolist()
     
    return features, ids

In [9]:
def data_generator(df):
    basis_df = df[["id","log_price"]]
    basis_df = basis_df.drop_duplicates()
    
    features = []
    ids = []
    
    for i in tqdm(np.unique(df["label"])):
        feat_cat, ids_cat = input_pipeline(i,df)
        df_tmp = pd.DataFrame({"features_"+str(i): feat_cat, "id": ids_cat})
        basis_df = pd.merge(basis_df, df_tmp, on = "id", how = "left")
    return basis_df
    

In [20]:
def data_generator(df):
    counter_overall_dummy = 0
    counter_overall_img = 0

    def input_pipeline(room = 0, df = df):
        ids = []
        features = []
        
        # FILTER DF
        df_room = df[df["label"] == room]
        
        # RESNET
        resnet = tf.keras.applications.resnet.ResNet50(include_top=False, weights='imagenet', pooling="avg", input_shape = (None,None,3))
        resnet_pre = keras.applications.resnet50.preprocess_input
        resnet.trainable = False
        counter_dummy = 0
        counter_img = 0
        for id in tqdm_notebook(np.unique(df["id"])):
            filter = df_room["id"] == id  

            try:
                l = []
                if filter.sum() == 0:
                    dummy_image = np.zeros((1,256,256,3))
                    dummy_image = resnet_pre(dummy_image)
                    dummy_image = resnet(dummy_image)
                    l.append(dummy_image)
                    counter_dummy += 1                
                else:
                    path_id = df_room["img_path"][filter]
                    for p in path_id:
                        img_tmp = plt.imread("data/images_resized/"+p)
                        img_tmp = np.expand_dims(img_tmp, axis = 0)
                        img_tmp = resnet_pre(img_tmp)
                        img_tmp = resnet(img_tmp)
                        l.append(img_tmp)
                        counter_img += 1
                l = np.stack(l)
                l = np.max(l, axis = 0)
                features.append(l)
                ids.append(id)
            except:
                dummy_image = np.zeros((1,256,256,3))
                dummy_image = resnet_pre(dummy_image)
                dummy_image = resnet(dummy_image)
                l.append(dummy_image)
                l = np.stack(l)
                l = np.max(l, axis = 0)
                features.append(l)
                ids.append(id)
                continue
        features = np.squeeze(np.stack(features))
        filter = np.nonzero(features.sum(axis = 0))[0]
        features = features[:,filter]
        print(len(filter), " features are nonzero.")
        features = features.tolist()
        print(counter_dummy, "dummy images were added.")
        return features, ids, counter_dummy, counter_img

    basis_df = df[["id","log_price"]]
    basis_df = basis_df.drop_duplicates()
    
    features = []
    ids = []
    
    for i in tqdm_notebook(np.unique(df["label"])):
        feat_cat, ids_cat, counter_dummy, counter_img = input_pipeline(i,df)
        df_tmp = pd.DataFrame({"features_"+str(i): feat_cat, "id": ids_cat})
        basis_df = pd.merge(basis_df, df_tmp, on = "id", how = "left")
        counter_overall_dummy += counter_dummy
        counter_overall_img += counter_img
    print(counter_overall_dummy)
    print(counter_overall_img)
    return basis_df
    

In [23]:
final_df = data_generator(df)

/var/folders/97/j215pw6x7sq158bvx1ktlhf80000gn/T/ipykernel_37481/1717659073.py:66: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(np.unique(df["label"])):


  0%|          | 0/6 [00:00<?, ?it/s]

2022-02-19 12:47:46.399099: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-02-19 12:47:46.399979: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



/var/folders/97/j215pw6x7sq158bvx1ktlhf80000gn/T/ipykernel_37481/1717659073.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for id in tqdm_notebook(np.unique(df["id"])):


...

  0%|          | 0/4356 [00:00<?, ?it/s]

1488  features are nonzero.
189 dummy images were added.


/var/folders/97/j215pw6x7sq158bvx1ktlhf80000gn/T/ipykernel_37481/1717659073.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for id in tqdm_notebook(np.unique(df["id"])):


  0%|          | 0/4356 [00:00<?, ?it/s]

1422  features are nonzero.
342 dummy images were added.
5583
56704


In [24]:
final_df.head()

,id,log_price,features_0.0,features_1.0,features_2.0,features_3.0,features_4.0,features_5.0
0,44077,4.174387,"[0.0, 0.0, 0.0, 0.008511412888765335, 0.0, 0.0...","[0.0, 0.0, 0.0005490838084369898, 0.0455342046...","[0.0, 0.0, 0.006581977009773254, 0.0, 0.053502...","[0.0, 0.0, 0.0, 0.0, 0.021639393642544746, 0.0...","[0.0, 0.0, 0.0, 0.05402378365397453, 0.0, 0.01...","[0.0, 0.0, 0.0014761326601728797, 0.0425817370..."
1,85156,4.143135,"[0.0, 0.0, 0.0, 0.02735969051718712, 0.0, 0.01...","[0.0, 0.0, 0.0, 0.0319686196744442, 0.0, 0.044...","[0.0, 0.0, 0.006581977009773254, 0.0, 0.053502...","[0.0, 0.0, 0.0, 0.0, 0.02309611067175865, 0.0,...","[0.0, 0.0, 0.0, 0.01392049714922905, 0.0, 0.01...","[0.0, 0.0, 0.0014761326601728797, 0.0425817370..."
2,159889,3.806662,"[0.0, 0.0, 0.0, 0.01579185761511326, 0.0, 0.04...","[0.0, 0.0, 0.0018560210010036826, 0.0118371285...","[0.0, 0.0, 0.006581977009773254, 0.0, 0.053502...","[0.0, 0.0, 0.0, 0.0, 0.006581977009773254, 0.0...","[0.0, 0.0, 0.0, 0.055553726851940155, 0.0, 0.0...","[0.0, 0.0, 0.0, 0.03788219392299652, 0.0, 0.01..."
3,162809,4.174387,"[0.0, 0.0, 0.0, 0.01077193021774292, 0.0, 0.04...","[0.0, 0.0, 0.0015976609429344535, 0.0854418128...","[0.0, 0.0, 0.006581977009773254, 0.0, 0.053502...","[0.0, 0.0, 0.0, 0.0, 0.006581977009773254, 0.0...","[0.0, 0.0, 0.0, 0.02611672878265381, 0.0, 0.01...","[0.0, 0.0, 0.0, 0.012000400573015213, 0.0, 0.0..."
4,165828,4.605170,"[0.0, 0.0, 0.0, 0.024556752294301987, 0.0, 0.0...","[0.0, 0.0, 0.0018485990585759282, 0.0464712753...","[0.0, 0.0014967878814786673, 0.088535755872726...","[0.0, 0.0, 0.0, 0.0, 0.006581977009773254, 0.0...","[0.0, 0.0, 0.0, 0.008630918338894844, 0.0, 0.0...","[0.0, 0.0, 4.3971813283860683e-05, 0.024929158..."


In [25]:
final_df.columns = ["id", "log_price", "bath", "bed", "dining", "hall", "kitchen", "living"]

In [30]:

# set columns for tabnet
bin_col = [col for col in X_train if np.isin(X_train[col].unique(), [0, 1]).all()]
num_col = [col for col in X_train if ~np.isin(X_train[col].unique(), [0, 1]).all()]
col_names = bin_col + num_col
feature_columns = []
col_names.remove("id")
for col in col_names:
    feature_columns.append(tf.feature_column.numeric_column(col))

In [33]:
# merge data for tabnet and images to get consistent ids
data_df_train = pd.merge(final_df, X_train, on="id", how="inner")
data_df_train.dropna(inplace = True)
print("TRAIN SHAPE: ", data_df_train.shape)

data_df_test = pd.merge(final_df, X_test, on="id", how="inner")
data_df_test.dropna(inplace = True)
print("TEST SHAPE: ", data_df_test.shape)

data_df_val = pd.merge(final_df, X_val, on="id", how="inner")
data_df_val.dropna(inplace = True)
print("VAL SHAPE: ", data_df_val.shape)

# extract consistent columns
X_train_tab = data_df_train.filter(col_names)
X_test_tab = data_df_test.filter(col_names)
X_val_tab = data_df_val.filter(col_names)

data_df_train.drop(col_names, axis = 1, inplace = True)
data_df_test.drop(col_names, axis = 1, inplace = True)
data_df_val.drop(col_names, axis = 1, inplace = True)

# extract consistent price
y_train = data_df_train.pop("log_price")
y_test = data_df_test.pop("log_price")
y_val = data_df_val.pop("log_price")

# drop id
X_train = data_df_train.drop("id", axis = 1)
X_test = data_df_test.drop("id", axis = 1)
X_val = data_df_val.drop("id", axis = 1)


TRAIN SHAPE:  (2735, 77)
TEST SHAPE:  (853, 77)
VAL SHAPE:  (681, 77)


In [34]:
def transform(ds):
    # tabnet
    features = tf.unstack(ds["features"])
    features = dict(zip(col_names, features))

    # images
    bath = tf.unstack(ds["bath"])
    bed = tf.unstack(ds["bed"])
    dining = tf.unstack(ds["dining"])
    hall = tf.unstack(ds["hall"])
    kitchen = tf.unstack(ds["kitchen"])
    living = tf.unstack(ds["living"])
    
    prices = ds["price"]
    
    y = prices
    return (bath, bed, dining, hall, kitchen, living, features), y

In [35]:
train_size = int(X_train.shape[0] * 0.9)
batch_size = int(X_train.shape[0] * 0.1)

# X_train, X_test, y_train, y_test = train_test_split(features, prices, random_state = 123, test_size = 0.2)
# X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size=0.2, random_state=1) # 0.25 x 0.8 = 0.2

data_train = tf.data.Dataset.from_tensor_slices({"bath": np.squeeze(np.stack(X_train["bath"])),
                                                 "bed": np.squeeze(np.stack(X_train["bed"])),
                                                 "dining": np.squeeze(np.stack(X_train["dining"])),
                                                 "hall": np.squeeze(np.stack(X_train["hall"])),
                                                 "kitchen": np.squeeze(np.stack(X_train["kitchen"])),
                                                 "living": np.squeeze(np.stack(X_train["living"])),
                                                 "features": X_train_tab,
                                                 "price": y_train})
data_train = data_train.cache()
data_train = data_train.shuffle(6000, seed = 13)
train_dataset = data_train.take(len(y_train))
train_dataset = train_dataset.map(transform)
train_dataset = train_dataset.batch(batch_size)

data_test = tf.data.Dataset.from_tensor_slices({"bath": np.squeeze(np.stack(X_test["bath"])),
                                                 "bed": np.squeeze(np.stack(X_test["bed"])),
                                                 "dining": np.squeeze(np.stack(X_test["dining"])),
                                                 "hall": np.squeeze(np.stack(X_test["hall"])),
                                                 "kitchen": np.squeeze(np.stack(X_test["kitchen"])),
                                                 "living": np.squeeze(np.stack(X_test["living"])),
                                                 "features": X_test_tab,
                                                 "price": y_test})
data_test = data_test.cache()
test_dataset = data_test.take(len(y_test))
test_dataset = test_dataset.map(transform)
test_dataset = test_dataset.batch(batch_size)

data_val = tf.data.Dataset.from_tensor_slices({"bath": np.squeeze(np.stack(X_val["bath"])),
                                                 "bed": np.squeeze(np.stack(X_val["bed"])),
                                                 "dining": np.squeeze(np.stack(X_val["dining"])),
                                                 "hall": np.squeeze(np.stack(X_val["hall"])),
                                                 "kitchen": np.squeeze(np.stack(X_val["kitchen"])),
                                                 "living": np.squeeze(np.stack(X_val["living"])),
                                                 "features": X_val_tab,
                                                 "price": y_val})
data_val = data_val.cache()
val_dataset = data_val.take(len(y_val))
val_dataset = val_dataset.map(transform)
val_dataset = val_dataset.batch(batch_size)

In [36]:
from tensorflow.keras import backend
class weight_constr(tf.keras.constraints.Constraint):
  """Constrains weight tensors to be centered around `ref_value`."""

  def __init__(self):
    self.ref_value = 1

  def __call__(self, w):
    nonneg = w * tf.cast(tf.greater_equal(w, 0.), backend.floatx())
    sum_w = tf.reduce_sum(nonneg)
    nonneg_one = nonneg/sum_w
    return nonneg_one


In [37]:
# setup model
class Compound_model(tf.keras.Model):

  def __init__(self,feature_columns, dropout = 0, l2 = 0, nodes1 = 512, nodes2 = 1,
                 num_features=None,
                 feature_dim=32,
                 output_dim=32,
                 num_decision_steps=3,
                 relaxation_factor=1.5,
                 sparsity_coefficient=1e-5,
                 norm_type='group',
                 batch_momentum=0.98,
                 virtual_batch_size=None,
                 num_groups=1,
                 epsilon=1e-5):
    super().__init__()
    # TabNet
    self.tabnet = tabnet.TabNet(feature_columns=feature_columns,
                          num_features=num_features,
                          feature_dim=feature_dim,
                          output_dim=output_dim,
                          num_decision_steps=num_decision_steps,
                          relaxation_factor=relaxation_factor,
                          sparsity_coefficient=sparsity_coefficient,
                          norm_type=norm_type,
                          batch_momentum=batch_momentum,
                          virtual_batch_size=virtual_batch_size,
                          num_groups=num_groups,
                          epsilon=epsilon)
   
    
    # bathroom
    self.bn1_bath = tf.keras.layers.BatchNormalization()
    self.drop1_bath = tf.keras.layers.Dropout(dropout)
    self.dense1_bath = tf.keras.layers.Dense(nodes1, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.bn2_bath = tf.keras.layers.BatchNormalization()
    self.drop2_bath = tf.keras.layers.Dropout(dropout)
    #self.dense2_bath = tf.keras.layers.Dense(nodes2, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.dense2_bath = tf.keras.layers.Dense(nodes2, kernel_regularizer = keras.regularizers.l2(l2))
 
    # bedroom
    self.bn1_bed = tf.keras.layers.BatchNormalization()
    self.drop1_bed = tf.keras.layers.Dropout(dropout)
    self.dense1_bed = tf.keras.layers.Dense(nodes1, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.bn2_bed = tf.keras.layers.BatchNormalization()
    self.drop2_bed = tf.keras.layers.Dropout(dropout)
    #self.dense2_bed = tf.keras.layers.Dense(nodes2, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.dense2_bed = tf.keras.layers.Dense(nodes2, kernel_regularizer = keras.regularizers.l2(l2))
       
      # dining
    self.bn1_dining = tf.keras.layers.BatchNormalization()
    self.drop1_dining = tf.keras.layers.Dropout(dropout)
    self.dense1_dining = tf.keras.layers.Dense(nodes1, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.bn2_dining = tf.keras.layers.BatchNormalization()
    self.drop2_dining = tf.keras.layers.Dropout(dropout)
    #self.dense2_dining = tf.keras.layers.Dense(nodes2, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.dense2_dining = tf.keras.layers.Dense(nodes2, kernel_regularizer = keras.regularizers.l2(l2))
   
    # hall
    self.bn1_hall = tf.keras.layers.BatchNormalization()
    self.drop1_hall = tf.keras.layers.Dropout(dropout)
    self.dense1_hall = tf.keras.layers.Dense(nodes1, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.bn2_hall = tf.keras.layers.BatchNormalization()
    self.drop2_hall = tf.keras.layers.Dropout(dropout)
    #self.dense2_hall = tf.keras.layers.Dense(nodes2, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.dense2_hall = tf.keras.layers.Dense(nodes2, kernel_regularizer = keras.regularizers.l2(l2))
    
      # kitchen
    self.bn1_kitchen = tf.keras.layers.BatchNormalization()
    self.drop1_kitchen = tf.keras.layers.Dropout(dropout)
    self.dense1_kitchen = tf.keras.layers.Dense(nodes1, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.bn2_kitchen = tf.keras.layers.BatchNormalization()
    self.drop2_kitchen = tf.keras.layers.Dropout(dropout)
    #self.dense2_kitchen = tf.keras.layers.Dense(nodes2, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.dense2_kitchen = tf.keras.layers.Dense(nodes2, kernel_regularizer = keras.regularizers.l2(l2))
  
      # livingroom
    self.bn1_living = tf.keras.layers.BatchNormalization()
    self.drop1_living = tf.keras.layers.Dropout(dropout)
    self.dense1_living = tf.keras.layers.Dense(nodes1, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.bn2_living = tf.keras.layers.BatchNormalization()
    self.drop2_living = tf.keras.layers.Dropout(dropout)
    #self.dense2_living = tf.keras.layers.Dense(nodes2, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.dense2_living = tf.keras.layers.Dense(nodes2, kernel_regularizer = keras.regularizers.l2(l2))
    
    # bring all images together
    self.bn_img = tf.keras.layers.BatchNormalization()
    self.drop_img = tf.keras.layers.Dropout(dropout)
    self.dense_img = tf.keras.layers.Dense(1, activation = tf.nn.relu)
        
    # final prediction
    self.bn_final = tf.keras.layers.BatchNormalization()
    self.drop_final = tf.keras.layers.Dropout(dropout)
    self.dense_final = tf.keras.layers.Dense(1, kernel_constraint =weight_constr())
    
  def call(self, inputs, training = None):
    
    # bathroom
    x = self.bn1_bath(inputs[0])
    x = self.drop1_bath(x)
    x = self.dense1_bath(x)
    x = self.bn2_bath(x)
    x = self.drop2_bath(x)
    bath_out = self.dense2_bath(x)
    
    
    # bedroom
    x = self.bn1_bed(inputs[1])
    x = self.drop1_bed(x)
    x = self.dense1_bed(x)
    x = self.bn2_bed(x)
    x = self.drop2_bed(x)
    bed_out = self.dense2_bed(x)
    
    # diningroom
    x = self.bn1_dining(inputs[2])
    x = self.drop1_dining(x)
    x = self.dense1_dining(x)
    x = self.bn2_dining(x)
    x = self.drop2_dining(x)
    dining_out = self.dense2_dining(x)
    
    # hallroom
    x = self.bn1_hall(inputs[3])
    x = self.drop1_hall(x)
    x = self.dense1_hall(x)
    x = self.bn2_hall(x)
    x = self.drop2_hall(x)
    hall_out = self.dense2_hall(x)
    
    # kitchen
    x = self.bn1_kitchen(inputs[4])
    x = self.drop1_kitchen(x)
    x = self.dense1_kitchen(x)
    x = self.bn2_kitchen(x)
    x = self.drop2_kitchen(x)
    kitchen_out = self.dense2_kitchen(x)
    
    # livingroom
    x = self.bn1_living(inputs[5])
    x = self.drop1_living(x)
    x = self.dense1_living(x)
    x = self.bn2_living(x)
    x = self.drop2_living(x)
    living_out = self.dense2_living(x)
    
    
    # tabnet
    self.activations = self.tabnet(inputs[6], training=True)

    # join images
    out_img = tf.keras.layers.concatenate([bath_out, bed_out, dining_out, hall_out, kitchen_out, living_out])
    out_img = self.bn_img(out_img)
    out_img = self.drop_img(out_img)

    # join
    out = tf.keras.layers.concatenate([out_img,self.activations])
    out = self.bn_final(out)
    out = self.drop_final(out)
    return self.dense_final(out)

In [38]:
def R_squared(y, y_pred):
  residual = tf.reduce_sum(tf.square(tf.subtract(y, y_pred)))
  total = tf.reduce_sum(tf.square(tf.subtract(y, tf.reduce_mean(y))))
  r2 = tf.subtract(1.0, tf.math.divide(residual, total))
  
  return r2

In [41]:
model = Compound_model(feature_columns = feature_columns,
                                output_dim=40, feature_dim=50, num_groups=1,
                                num_decision_steps=2, relaxation_factor=3.5,
                                sparsity_coefficient=1e-05, dropout= 0, nodes1 = 64, nodes2 = 1, l2 = 0)
lr = tf.keras.optimizers.schedules.ExponentialDecay(0.01, decay_steps=100, decay_rate=0.95, staircase=False)
#lr = 0.01
optimizer = tf.keras.optimizers.Adam(lr)
model.compile(optimizer, loss="mse", metrics=R_squared)


model.fit(train_dataset, validation_data = test_dataset, epochs=100, verbose=1)

[TabNet]: 10 features will be used for decision steps.


2022-02-19 13:43:13.926066: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-02-19 13:43:13.935412: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/100


NotImplementedError: in user code:

    /opt/homebrew/Caskroom/miniforge/base/envs/tensorflow_m1/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:855 train_function  *
        return step_function(self, iterator)
    /var/folders/97/j215pw6x7sq158bvx1ktlhf80000gn/T/ipykernel_37481/1185023008.py:149 call  *
        self.activations = self.tabnet(inputs[6], training=True)
    /opt/homebrew/Caskroom/miniforge/base/envs/tensorflow_m1/lib/python3.9/site-packages/tabnet/tabnet.py:224 call  *
        output_aggregated = tf.zeros([batch_size, self.output_dim])
    /opt/homebrew/Caskroom/miniforge/base/envs/tensorflow_m1/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:206 wrapper  **
        return target(*args, **kwargs)
    /opt/homebrew/Caskroom/miniforge/base/envs/tensorflow_m1/lib/python3.9/site-packages/tensorflow/python/ops/array_ops.py:2911 wrapped
        tensor = fun(*args, **kwargs)
    /opt/homebrew/Caskroom/miniforge/base/envs/tensorflow_m1/lib/

In [125]:
# setup model
class Tabnet_img(tf.keras.Model):

  def __init__(self,feature_columns,
                 num_features=None,
                 feature_dim=32,
                 output_dim=32,
                 num_decision_steps=3,
                 relaxation_factor=1.5,
                 sparsity_coefficient=1e-5,
                 norm_type='group',
                 batch_momentum=0.98,
                 virtual_batch_size=None,
                 num_groups=1,
                 epsilon=1e-5):
    super().__init__()
    
    # image input
    self.resnet_pre = keras.applications.resnet50.preprocess_input
    self.resnet = tf.keras.applications.resnet.ResNet50(include_top=False, weights='imagenet', pooling="avg", input_shape = (None,None,3))
    self.resnet.trainable = False
    
    self.bn1 = tf.keras.layers.BatchNormalization()
    self.drop = tf.keras.layers.Dropout(0)
    self.dense1 = tf.keras.layers.Dense(512, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(0))
    self.dense2 = tf.keras.layers.Dense(32, activation=tf.nn.relu)
    
    self.tabnet = tabnet.TabNet(feature_columns=feature_columns,
                             num_features=num_features,
                             feature_dim=feature_dim,
                             output_dim=output_dim,
                             num_decision_steps=num_decision_steps,
                             relaxation_factor=relaxation_factor,
                             sparsity_coefficient=sparsity_coefficient,
                             norm_type=norm_type,
                             batch_momentum=batch_momentum,
                             virtual_batch_size=virtual_batch_size,
                             num_groups=num_groups,
                             epsilon=epsilon)
   
    self.dense_out = tf.keras.layers.Dense(1)
  def call(self, inputs, training = None):
    # images
    x = self.resnet_pre(inputs[0])
    x = self.resnet(x)
    x = self.bn1(inputs[0])
    x = self.drop(x)
    x = self.dense1(x)
    out1 = self.dense2(x)
    
    # tabnet
    self.activations = self.tabnet(inputs[1], training=training)
    
    # join
    out = tf.keras.layers.concatenate([out1,self.activations])
    return self.dense_out(out)

In [126]:
# store id and features in dataframe
features = [bag_features[x] for x in range(bag_features.shape[0])]
bag_df = pd.DataFrame({"id": ids, "features": features})

In [128]:
X_train["price"] = y_train.values
X_test["price"] = y_test.values

In [129]:
# merge data for tabnet and images to get consistent ids
data_df = pd.merge(bag_df, X_train, on="id", how="inner")
data_df_test = pd.merge(bag_df, X_test, on="id", how="inner")

# extract consistent columns
X_train_new = data_df.filter(col_names)
X_test_new = data_df_test.filter(col_names)

# extract consistent price
y_train_new = data_df["price"]
y_test_new = data_df_test["price"]

In [130]:
# # drop columns that are no longer necessary
# X_train_new.drop("id", axis =1, inplace = True)
# X_test_new.drop("id", axis =1, inplace = True)


In [131]:

# set columns for tabnet
bin_col = [col for col in X_train_new if np.isin(X_train_new[col].unique(), [0, 1]).all()]
num_col = [col for col in X_train_new if ~np.isin(X_train_new[col].unique(), [0, 1]).all()]
col_names = bin_col + num_col
feature_columns = []

for col in col_names:
    feature_columns.append(tf.feature_column.numeric_column(col))

In [132]:
def transform(ds):
    features = tf.unstack(ds["features"])
    prices = ds["price"]
    
    img = ds["images"]
    x = dict(zip(col_names, features))
    y = prices
    return (img, x), y

In [133]:
# make tf dataset
data_train = tf.data.Dataset.from_tensor_slices({"images": np.stack(data_df["features"]),"features": X_train_new, "price": y_train_new})
data_train = data_train.shuffle(6000, seed = 13)
train_dataset = data_train.take(len(data_df))
train_dataset = train_dataset.map(transform)
train_dataset = train_dataset.batch(60)

data_test= tf.data.Dataset.from_tensor_slices({"images": np.stack(data_df_test["features"]),"features": X_test_new, "price": y_test_new})
test_dataset = data_test.take(len(data_df_test))
test_dataset = test_dataset.map(transform)
test_dataset = test_dataset.batch(60)

In [134]:
len(col_names)

42

In [135]:
model = Tabnet_img(feature_columns = feature_columns, feature_dim = 42, output_dim =32)

[TabNet]: 10 features will be used for decision steps.


In [136]:
def R_squared(y, y_pred):
  residual = tf.reduce_sum(tf.square(tf.subtract(y, y_pred)))
  total = tf.reduce_sum(tf.square(tf.subtract(y, tf.reduce_mean(y))))
  r2 = tf.subtract(1.0, tf.math.divide(residual, total))
  
  return r2

In [137]:
lr = tf.keras.optimizers.schedules.ExponentialDecay(0.01, decay_steps=100, decay_rate=0.9, staircase=False)
optimizer = tf.keras.optimizers.Adam(lr)
model.compile(optimizer, loss="mse", metrics=R_squared)


model.fit(train_dataset, validation_data = test_dataset, epochs=100, verbose=1)

Epoch 1/100


2022-02-02 13:04:46.158074: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


83/83 [==============================] - ETA: 0s - loss: 3.9533 - R_squared: -6.4239

2022-02-02 13:04:53.829804: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


83/83 [==============================] - 10s 93ms/step - loss: 3.9533 - R_squared: -6.4239 - val_loss: 1.0087 - val_R_squared: -0.9259
Epoch 2/100
83/83 [==============================] - 7s 86ms/step - loss: 0.4514 - R_squared: 0.2395 - val_loss: 0.5291 - val_R_squared: 0.0300
Epoch 3/100
83/83 [==============================] - 7s 86ms/step - loss: 0.4083 - R_squared: 0.3112 - val_loss: 0.4312 - val_R_squared: 0.2173
Epoch 4/100
83/83 [==============================] - 7s 86ms/step - loss: 0.3830 - R_squared: 0.3599 - val_loss: 0.3467 - val_R_squared: 0.3688
Epoch 5/100
83/83 [==============================] - 7s 86ms/step - loss: 0.3592 - R_squared: 0.3978 - val_loss: 0.3668 - val_R_squared: 0.3346
Epoch 6/100
83/83 [==============================] - 7s 86ms/step - loss: 0.3326 - R_squared: 0.4340 - val_loss: 0.3643 - val_R_squared: 0.3371
Epoch 7/100
83/83 [==============================] - 7s 87ms/step - loss: 0.3094 - R_squared: 0.4867 - val_loss: 0.3168 - val_R_squared: 0.4097
E

# Try with collage data

In [57]:
# Load data
X_train, X_test, y_train, y_test = load_data(drop_id=False, skip = False)

------------------------------
Selecting Features data...
------------------------------
------------------------------
Loading data...
------------------------------


/Users/dmnk/OneDrive - stud.uni-goettingen.de/Dokumente/3. Semester/SeminarDL/DubAir/load_data.py:73: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  price = price.str.replace("$","")


Data loaded.


/Users/dmnk/OneDrive - stud.uni-goettingen.de/Dokumente/3. Semester/SeminarDL/DubAir/load_data.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listings["host_location_country"][fil] = str(i)
/Users/dmnk/OneDrive - stud.uni-goettingen.de/Dokumente/3. Semester/SeminarDL/DubAir/load_data.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listings["host_location_country"][fil] = str(i)
/Users/dmnk/OneDrive - stud.uni-goettingen.de/Dokumente/3. Semester/SeminarDL/DubAir/load_data.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation:

Data cleansed.
Imputation done. No NaN's are left in the data.
Further Modifications are done.
Text and Open Street Data generated.
Image data loaded.
Due to insignificant t-tests we dropped:
['Bathtub', 'Bed linens', 'Breakfast', 'Cleaning before checkout', 'Dishwasher', 'Elevator', 'Hair dryer', 'Indoor fireplace', 'Private entrance', 'Security cameras on property', 'Single level home', 'Stoves_available', 'Refridgerator_available', 'Body_soap_available', 'Garden_backyard_available', 'Children_Entertainment', 'Workspace', 'Gym_available', 'Coffee_machine_available', 'Dryer_available', 'Washer_available', 'Hot_tub_available', 'Pool_available', 'Wifi_available', 'AC_available', 'heating_available', 'Kitchen_available', 'Safe_available', 'Water_location', 'sound_system_available', 'TV_available', 'Outdoor_stuff', 'Game_consoles', 'Baby_friendly', 'Special_stuff', 'neighbourhood_cleansed_Dn Laoghaire-Rathdown', 'property_type_Entire guesthouse', 'property_type_Entire loft', 'property_typ

In [58]:

images = np.load("data/images_collage.npy", allow_pickle = True)
ids = np.load("data/ids_collage.npy").astype(int)
price = np.load("data/price_collage.npy", allow_pickle = True)

In [59]:
# setup model
class Tabnet_img(tf.keras.Model):

  def __init__(self,feature_columns,
                 num_features=None,
                 feature_dim=32,
                 output_dim=32,
                 num_decision_steps=3,
                 relaxation_factor=1.5,
                 sparsity_coefficient=1e-5,
                 norm_type='group',
                 batch_momentum=0.98,
                 virtual_batch_size=None,
                 num_groups=1,
                 epsilon=1e-5):
    super().__init__()
    
    # image input
    self.resnet_pre = keras.applications.resnet50.preprocess_input
    self.resnet = tf.keras.applications.resnet.ResNet50(include_top=False, weights='imagenet', pooling="avg", input_shape = (None,None,3))
    self.resnet.trainable = False
    
    self.bn1 = tf.keras.layers.BatchNormalization()
    self.drop = tf.keras.layers.Dropout(0)
    self.dense1 = tf.keras.layers.Dense(512, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(0))
    self.dense2 = tf.keras.layers.Dense(32, activation=tf.nn.relu)
    
    self.tabnet = tabnet.TabNet(feature_columns=feature_columns,
                             num_features=num_features,
                             feature_dim=feature_dim,
                             output_dim=output_dim,
                             num_decision_steps=num_decision_steps,
                             relaxation_factor=relaxation_factor,
                             sparsity_coefficient=sparsity_coefficient,
                             norm_type=norm_type,
                             batch_momentum=batch_momentum,
                             virtual_batch_size=virtual_batch_size,
                             num_groups=num_groups,
                             epsilon=epsilon)
   
    self.dense_out = tf.keras.layers.Dense(1)
  def call(self, inputs, training = None, train_resnet = False):
    if train_resnet:
      self.resnet.trainable = True
    # images
    x = tf.cast(inputs[0], dtype=tf.float32)
    x = self.resnet_pre(inputs[0])
    x = self.resnet(x)
    x = self.bn1(inputs[0])
    x = self.drop(x)
    x = self.dense1(x)
    out1 = self.dense2(x)
    
    # tabnet
    self.activations = self.tabnet(inputs[1], training=training)
    
    # join
    out = tf.keras.layers.concatenate([out1,self.activations])
    return self.dense_out(out)

In [60]:
# store id and images in dataframe
features = [images[x] for x in range(images.shape[0])]
images_df = pd.DataFrame({"id": ids, "images": features})

In [61]:
# rename columns (for now)
X_train.columns = X_train.columns.str.replace(" ","_")
X_test.columns = X_test.columns.str.replace(" ","_")

In [44]:

# set columns for tabnet
bin_col = [col for col in X_train if np.isin(X_train[col].unique(), [0, 1]).all()]
num_col = [col for col in X_train if ~np.isin(X_train[col].unique(), [0, 1]).all()]
col_names = bin_col + num_col
col_names.remove("id")
feature_columns = []

for col in col_names:
    feature_columns.append(tf.feature_column.numeric_column(col))

In [45]:
X_train["price"] = y_train.values
X_test["price"] = y_test.values

In [46]:
# merge data for tabnet and images to get consistent ids
data_df = pd.merge(images_df, X_train, on="id", how="inner")
data_df_test = pd.merge(images_df, X_test, on="id", how="inner")

# extract consistent columns
X_train_new = data_df.filter(col_names)
X_test_new = data_df_test.filter(col_names)

# extract consistent price
y_train_new = data_df["price"]
y_test_new = data_df_test["price"]

In [56]:
# # drop columns that are no longer necessary
X_train_new.drop("id", axis =1, inplace = True)
X_test_new.drop("id", axis =1, inplace = True)


KeyError: "['id'] not found in axis"

In [48]:
def transform(ds):
    features = tf.unstack(ds["features"])
    prices = ds["price"]
    
    img = ds["images"]
    x = dict(zip(col_names, features))
    y = prices
    return (img, x), y

In [49]:
# make tf dataset
data_train = tf.data.Dataset.from_tensor_slices({"images": np.stack(data_df["images"]),"features": X_train_new, "price": y_train_new})
data_train = data_train.shuffle(6000, seed = 13)
train_dataset = data_train.take(len(data_df))
train_dataset = train_dataset.map(transform)
train_dataset = train_dataset.batch(60)

data_test= tf.data.Dataset.from_tensor_slices({"images": np.stack(data_df_test["images"]),"features": X_test_new, "price": y_test_new})
test_dataset = data_test.take(len(data_df_test))
test_dataset = test_dataset.map(transform)
test_dataset = test_dataset.batch(60)

In [55]:
len(col_names)

42

In [52]:
model = Tabnet_img(feature_columns = feature_columns, feature_dim = 42, output_dim =32)

[TabNet]: 10 features will be used for decision steps.


In [53]:
def R_squared(y, y_pred):
  residual = tf.reduce_sum(tf.square(tf.subtract(y, y_pred)))
  total = tf.reduce_sum(tf.square(tf.subtract(y, tf.reduce_mean(y))))
  r2 = tf.subtract(1.0, tf.math.divide(residual, total))
  
  return r2

In [54]:
lr = tf.keras.optimizers.schedules.ExponentialDecay(0.01, decay_steps=100, decay_rate=0.9, staircase=False)
optimizer = tf.keras.optimizers.Adam(lr)
model.compile(optimizer, loss="mse", metrics=R_squared)


model.fit(train_dataset, validation_data = test_dataset, epochs=100, verbose=1)

Epoch 1/100


TypeError: in user code:

    /opt/homebrew/Caskroom/miniforge/base/envs/tensorflow_m1/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:855 train_function  *
        return step_function(self, iterator)
    /var/folders/97/j215pw6x7sq158bvx1ktlhf80000gn/T/ipykernel_1490/3916806783.py:49 call  *
        x = self.bn1(inputs[0])
    /opt/homebrew/Caskroom/miniforge/base/envs/tensorflow_m1/lib/python3.9/site-packages/tensorflow/python/keras/engine/base_layer.py:1030 __call__  **
        outputs = call_fn(inputs, *args, **kwargs)
    /opt/homebrew/Caskroom/miniforge/base/envs/tensorflow_m1/lib/python3.9/site-packages/tensorflow/python/keras/layers/normalization.py:772 call
        outputs = self._fused_batch_norm(inputs, training=training)
    /opt/homebrew/Caskroom/miniforge/base/envs/tensorflow_m1/lib/python3.9/site-packages/tensorflow/python/keras/layers/normalization.py:625 _fused_batch_norm
        output, mean, variance = control_flow_util.smart_cond(
    /opt/ho